In [1]:
import facebook
from datetime import datetime, timedelta
import pandas as pd

In [21]:
access_token = "EAACEdEose0cBABZBZC6BAgg8MoLZASKapZCygmkuWAMGxskMLfh17KX22XI1pneXX78I2c87dxJrEtOrsnC6dmFbGeHjg4kJyPp3twe94m4P5nNEQCZBRcFFZAzxFDsgk4i7TJU6gHkWLXoijkFcBEUOaFlZBB4YMjH7NEt8OUDKZCl8KeUaUs0tpkKvds4VjgcZD"
graph = facebook.GraphAPI(access_token, version="2.10")

In [3]:
"""Retrive post_id of a page which posted with chopped_days"""
def retrive_postsid(page_id, chopped_days):
    page_posts = graph.get_object('/'+str(page_id)+'/posts?')
    page_posts_used = []
    post_items = []
    while True:
        for post in page_posts['data']:
            created_time = datetime.strptime(post['created_time'][:19], '%Y-%m-%dT%H:%M:%S')
            chopped_time = datetime.now() - timedelta(days=chopped_days)
            if created_time>chopped_time:
                post_item = dict()
                post_item['post_id'] = post['id']
                post_item['page_id'] = page_id
                post_item['created_time'] = post['created_time'][:10]
                post_items.append(post_item)
                page_posts_used.append(post['id'])
            else:
                break
        if created_time<chopped_time:
            break
        if 'next' in page_posts['paging'].keys():
            after_key = page_posts['paging']['cursors']['after']
            page_posts = graph.get_object('/'+str(page_id)+'/posts?after='+after_key)
        else:
            break
    
    return page_posts_used, pd.DataFrame(post_items)

In [ ]:
def retrive_reactions(posts_id, page_id):
    reaction_items = []
    i = 1
    for post_id in posts_id:
        print('retriving reactions')
        print(i, '/', len(posts_id) ,'At post_id: ', post_id, )
        i = i + 1
        post_reactions = graph.get_object('/'+str(post_id)+'/reactions')
        if bool(post_reactions['data']):    
            while True:
                for reaction in post_reactions['data']:
                    reaction_item = dict()
                    reaction_item['user_id'] = reaction['id']
                    reaction_item['post_id'] = post_id
                    reaction_item['page_id'] = page_id
                    reaction_item['reaction_type'] = reaction['type']
                    reaction_items.append(reaction_item)
                if 'next' in post_reactions['paging']:
                    after_key = post_reactions['paging']['cursors']['after']
                    post_reactions = graph.get_object('/'+str(post_id)+'/reactions?after='+str(after_key))
                else:
                    break
    print('finish retriving reactions!')
    return pd.DataFrame(reaction_items)  

In [ ]:
def retrive_comments(posts_id, page_id):    
    comment_items = []
    i=1
    for post_id in posts_id:
        print('retriving comments')
        print(i, '/', len(posts_id) ,'At post_id: ', post_id, )
        i = i + 1
        post_comments = graph.get_object('/'+str(post_id)+'/comments')
        if bool(post_comments['data']):
            while True:
                for comment in post_comments['data']:
                    comment_item = dict()
                    comment_item['user_id'] = comment['from']['id']
                    comment_item['post_id'] = post_id
                    comment_item['page_id'] = '366743047016721'
                    comment_item['comment_body'] = comment['message']
                    comment_items.append(comment_item)
                if 'next' in post_comments['paging']:
                    after_key = post_comments['paging']['cursors']['after']
                    post_comments = graph.get_object('/'+str(post_id)+'/comments?after='+str(after_key))
                else:
                    break
    print('finish retriving comments!')
    return pd.DataFrame(comment_items) 

In [6]:
def retrive_pageid(page_name):
    return graph.get_object('/'+str(page_name))['id']

In [7]:
def retrive_data(page_name, chopped_days):
    from pathlib import Path
    comments_dir = Path('/data/comments/'+page_name+'.csv')
    page_id = retrive_pageid(page_name)
    posts_id, db_posts = retrive_postsid(page_id, chopped_days)
    db_posts.to_csv('./data/posts/'+page_name+'.csv')
    if comments_dir.exists():
        pass
    else:
        db_comments = retrive_comments(posts_id, page_id)
        db_comments.to_csv('./data/comments/'+page_name+'.csv')
    db_reactions = retrive_reactions(posts_id, page_id)
    db_reactions.to_csv('./data/reactions/'+page_name+'.csv') 
    return db_comments, db_reactions, db_posts

In [8]:
def retrive_performance(posts_id, page_id):
    rows = []
    for post_id in posts_id:
        if 'shares' in graph.get_object('/'+str(post_id)+'/?fields=shares'):
            num_reactions = graph.get_object('/'+str(post_id)+'/reactions?summary=1')['summary']['total_count'] 
            num_comments = graph.get_object('/'+str(post_id)+'/comments?summary=1')['summary']['total_count'] 
            num_shares = graph.get_object('/'+str(post_id)+'/?fields=shares')['shares']['count'] 
            row = dict()
            row['num_reactions'] = num_reactions
            row['num_comments'] = num_comments
            row['num_shares'] = num_shares
            row['page_id'] = page_id
            row['post_id'] = post_id
            rows.append(row)
    return pd.DataFrame(rows)

In [9]:
def retrive_overalldata(page_name, chopped_days):
    page_id = retrive_pageid(page_name)
    posts_id, _ = retrive_postsid(page_id, chopped_days)
    db_performance = retrive_performance(posts_id, page_id)
    db_performance.to_csv('./data/summary/'+page_name+'.csv')

In [10]:
def retrive_facebook_page(page_name):  
    _, _, _ = retrive_data(page_name, 180)
    retrive_overalldata(page_name, 90)
    db_summary_temp = pd.read_csv('./data/summary/'+page_name+'.csv')
    db_posts_temp = pd.read_csv('./data/posts/'+page_name+'.csv')
    db_posts_temp = db_posts_temp[['post_id', 'created_time']]
    temp = pd.merge(db_summary_temp, db_posts_temp, how='inner', on='post_id')
    temp.iloc[:,1:].to_csv('./data/summary2/'+page_name+'.csv')

In [13]:
retrive_facebook_page('badtaste.donteat')

retriving comments
1 / 18 At post_id:  208067196022874_858050647691189
retriving comments
2 / 18 At post_id:  208067196022874_856182577877996
retriving comments
3 / 18 At post_id:  208067196022874_848590148637239
retriving comments
4 / 18 At post_id:  208067196022874_845509998945254
retriving comments
5 / 18 At post_id:  208067196022874_844939405668980
retriving comments
6 / 18 At post_id:  208067196022874_842411059255148
retriving comments
7 / 18 At post_id:  208067196022874_841836162645971
retriving comments
8 / 18 At post_id:  208067196022874_841709565991964
retriving comments
9 / 18 At post_id:  208067196022874_841346619361592
retriving comments
10 / 18 At post_id:  208067196022874_841333099362944
retriving comments
11 / 18 At post_id:  208067196022874_840074852822102
retriving comments
12 / 18 At post_id:  208067196022874_839278856235035
retriving comments
13 / 18 At post_id:  208067196022874_839276296235291
retriving comments
14 / 18 At post_id:  208067196022874_823824037780517
r

In [16]:
retrive_facebook_page('raosaidak')

retriving comments
1 / 89 At post_id:  1723496894570900_1974952409425346
retriving comments
2 / 89 At post_id:  1723496894570900_1974785292775391
retriving comments
3 / 89 At post_id:  1723496894570900_1973790762874844
retriving comments
4 / 89 At post_id:  1723496894570900_1972827679637819
retriving comments
5 / 89 At post_id:  1723496894570900_1972799719640615
retriving comments
6 / 89 At post_id:  1723496894570900_1970197323234188
retriving comments
7 / 89 At post_id:  1723496894570900_1969957099924877
retriving comments
8 / 89 At post_id:  1723496894570900_1967242130196374
retriving comments
9 / 89 At post_id:  1723496894570900_1967209740199613
retriving comments
10 / 89 At post_id:  1723496894570900_1964956327091621
retriving comments
11 / 89 At post_id:  1723496894570900_1964816437105610
retriving comments
12 / 89 At post_id:  1723496894570900_1963701313883789
retriving comments
13 / 89 At post_id:  1723496894570900_1963222683931652
retriving comments
14 / 89 At post_id:  1723496

retriving reactions
23 / 89 At post_id:  1723496894570900_1954422291478358
retriving reactions
24 / 89 At post_id:  1723496894570900_1952766714977249
retriving reactions
25 / 89 At post_id:  1723496894570900_1951044538482800
retriving reactions
26 / 89 At post_id:  1723496894570900_1949264971994090
retriving reactions
27 / 89 At post_id:  1723496894570900_1948860282034559
retriving reactions
28 / 89 At post_id:  1723496894570900_1946218492298738
retriving reactions
29 / 89 At post_id:  1723496894570900_1946094772311110
retriving reactions
30 / 89 At post_id:  1723496894570900_1946018055652115
retriving reactions
31 / 89 At post_id:  1723496894570900_1944270145826906
retriving reactions
32 / 89 At post_id:  1723496894570900_1941449592775628
retriving reactions
33 / 89 At post_id:  1723496894570900_1940926699494584
retriving reactions
34 / 89 At post_id:  1723496894570900_1940652939521960
retriving reactions
35 / 89 At post_id:  1723496894570900_1937151236538797
retriving reactions
36 / 

In [13]:
retrive_facebook_page('gin100')

retriving comments
1 / 99 At post_id:  387766624666267_1376755972433989
retriving comments
2 / 99 At post_id:  387766624666267_1374997329276520
retriving comments
3 / 99 At post_id:  387766624666267_1374923529283900
retriving comments
4 / 99 At post_id:  387766624666267_1374817382627848
retriving comments
5 / 99 At post_id:  387766624666267_1373986069377646
retriving comments
6 / 99 At post_id:  387766624666267_1371546919621561
retriving comments
7 / 99 At post_id:  387766624666267_1371112506331669
retriving comments
8 / 99 At post_id:  387766624666267_1370208803088706
retriving comments
9 / 99 At post_id:  387766624666267_1369182373191349
retriving comments
10 / 99 At post_id:  387766624666267_1366058700170383
retriving comments
11 / 99 At post_id:  387766624666267_1363655507077369
retriving comments
12 / 99 At post_id:  387766624666267_1358277920948461
retriving comments
13 / 99 At post_id:  387766624666267_1355069491269304
retriving comments
14 / 99 At post_id:  387766624666267_1354

retriving reactions
14 / 99 At post_id:  387766624666267_1354137554695831
retriving reactions
15 / 99 At post_id:  387766624666267_1352616944847892
retriving reactions
16 / 99 At post_id:  387766624666267_1352449908197929
retriving reactions
17 / 99 At post_id:  387766624666267_1352362011540052
retriving reactions
18 / 99 At post_id:  387766624666267_1350882471688006
retriving reactions
19 / 99 At post_id:  387766624666267_1348707761905477
retriving reactions
20 / 99 At post_id:  387766624666267_1348083801967873
retriving reactions
21 / 99 At post_id:  387766624666267_1346771098765810
retriving reactions
22 / 99 At post_id:  387766624666267_1346076975501889
retriving reactions
23 / 99 At post_id:  387766624666267_1344231692353084
retriving reactions
24 / 99 At post_id:  387766624666267_1341866422589611
retriving reactions
25 / 99 At post_id:  387766624666267_1341124042663849
retriving reactions
26 / 99 At post_id:  387766624666267_1340046256104961
retriving reactions
27 / 99 At post_id

In [17]:
retrive_facebook_page('kinlagthailand')

retriving comments
1 / 131 At post_id:  266497693552828_746946325507960
retriving comments
2 / 131 At post_id:  266497693552828_744949435707649
retriving comments
3 / 131 At post_id:  266497693552828_743657462503513
retriving comments
4 / 131 At post_id:  266497693552828_740343889501537
retriving comments
5 / 131 At post_id:  266497693552828_738688416333751
retriving comments
6 / 131 At post_id:  266497693552828_736275476575045
retriving comments
7 / 131 At post_id:  266497693552828_733382826864310
retriving comments
8 / 131 At post_id:  266497693552828_733375046865088
retriving comments
9 / 131 At post_id:  266497693552828_732616390274287
retriving comments
10 / 131 At post_id:  266497693552828_732572423612017
retriving comments
11 / 131 At post_id:  266497693552828_732617326940860
retriving comments
12 / 131 At post_id:  266497693552828_732336796968913
retriving comments
13 / 131 At post_id:  266497693552828_731603367042256
retriving comments
14 / 131 At post_id:  266497693552828_711

retriving comments
114 / 131 At post_id:  266497693552828_671792369690023
retriving comments
115 / 131 At post_id:  266497693552828_671742223028371
retriving comments
116 / 131 At post_id:  266497693552828_671715153031078
retriving comments
117 / 131 At post_id:  266497693552828_671405439728716
retriving comments
118 / 131 At post_id:  266497693552828_671360019733258
retriving comments
119 / 131 At post_id:  266497693552828_670596909809569
retriving comments
120 / 131 At post_id:  266497693552828_670462903156303
retriving comments
121 / 131 At post_id:  266497693552828_670011069868153
retriving comments
122 / 131 At post_id:  266497693552828_669719106564016
retriving comments
123 / 131 At post_id:  266497693552828_669565036579423
retriving comments
124 / 131 At post_id:  266497693552828_669105096625417
retriving comments
125 / 131 At post_id:  266497693552828_668768363325757
retriving comments
126 / 131 At post_id:  266497693552828_668573323345261
retriving comments
127 / 131 At post_i

retriving reactions
94 / 131 At post_id:  266497693552828_675670529302207
retriving reactions
95 / 131 At post_id:  266497693552828_675581535977773
retriving reactions
96 / 131 At post_id:  266497693552828_675567452645848
retriving reactions
97 / 131 At post_id:  266497693552828_675364252666168
retriving reactions
98 / 131 At post_id:  266497693552828_674867682715825
retriving reactions
99 / 131 At post_id:  266497693552828_674115252791068
retriving reactions
100 / 131 At post_id:  266497693552828_674063432796250
retriving reactions
101 / 131 At post_id:  266497693552828_674017296134197
retriving reactions
102 / 131 At post_id:  266497693552828_673773362825257
retriving reactions
103 / 131 At post_id:  266497693552828_673671859502074
retriving reactions
104 / 131 At post_id:  266497693552828_673583182844275
retriving reactions
105 / 131 At post_id:  266497693552828_673562619512998
retriving reactions
106 / 131 At post_id:  266497693552828_673302579539002
retriving reactions
107 / 131 A

In [18]:

retrive_facebook_page('Enjoyeatingbyme')

retriving comments
1 / 81 At post_id:  320764744931291_569566213384475
retriving comments
2 / 81 At post_id:  320764744931291_567033416971088
retriving comments
3 / 81 At post_id:  320764744931291_565600293781067
retriving comments
4 / 81 At post_id:  320764744931291_563418500665913
retriving comments
5 / 81 At post_id:  320764744931291_561890404152056
retriving comments
6 / 81 At post_id:  320764744931291_561135227560907
retriving comments
7 / 81 At post_id:  320764744931291_560533457621084
retriving comments
8 / 81 At post_id:  320764744931291_558881544452942
retriving comments
9 / 81 At post_id:  320764744931291_558078907866539
retriving comments
10 / 81 At post_id:  320764744931291_557591334581963
retriving comments
11 / 81 At post_id:  320764744931291_557224564618640
retriving comments
12 / 81 At post_id:  320764744931291_554914441516319
retriving comments
13 / 81 At post_id:  320764744931291_554148574926239
retriving comments
14 / 81 At post_id:  320764744931291_554235448250885
r

retriving reactions
34 / 81 At post_id:  320764744931291_524331707907926
retriving reactions
35 / 81 At post_id:  320764744931291_523136994694064
retriving reactions
36 / 81 At post_id:  320764744931291_522079281466502
retriving reactions
37 / 81 At post_id:  320764744931291_521176041556826
retriving reactions
38 / 81 At post_id:  320764744931291_520232718317825
retriving reactions
39 / 81 At post_id:  320764744931291_519689281705502
retriving reactions
40 / 81 At post_id:  320764744931291_518125668528530
retriving reactions
41 / 81 At post_id:  320764744931291_517834341890996
retriving reactions
42 / 81 At post_id:  320764744931291_516156862058744
retriving reactions
43 / 81 At post_id:  320764744931291_515336135474150
retriving reactions
44 / 81 At post_id:  320764744931291_514451695562594
retriving reactions
45 / 81 At post_id:  320764744931291_513353275672436
retriving reactions
46 / 81 At post_id:  320764744931291_512718182402612
retriving reactions
47 / 81 At post_id:  3207647449

In [19]:
retrive_facebook_page('aroiyummy')

retriving comments
1 / 35 At post_id:  600475487009484_616522735404759
retriving comments
2 / 35 At post_id:  600475487009484_614954928894873
retriving comments
3 / 35 At post_id:  600475487009484_614713318919034
retriving comments
4 / 35 At post_id:  600475487009484_613792402344459
retriving comments
5 / 35 At post_id:  600475487009484_612828169107549
retriving comments
6 / 35 At post_id:  600475487009484_612098852513814
retriving comments
7 / 35 At post_id:  600475487009484_611327185924314
retriving comments
8 / 35 At post_id:  600475487009484_610722452651454
retriving comments
9 / 35 At post_id:  600475487009484_610538302669869
retriving comments
10 / 35 At post_id:  600475487009484_609930839397282
retriving comments
11 / 35 At post_id:  600475487009484_609707046086328
retriving comments
12 / 35 At post_id:  600475487009484_609699122753787
retriving comments
13 / 35 At post_id:  600475487009484_609657976091235
retriving comments
14 / 35 At post_id:  600475487009484_609054032818296
r

In [20]:
retrive_facebook_page('Unlucky2BDelicious')

retriving comments
1 / 73 At post_id:  491309534239885_1434882933215869
retriving comments
2 / 73 At post_id:  491309534239885_1430556366981859
retriving comments
3 / 73 At post_id:  491309534239885_1430366870334142
retriving comments
4 / 73 At post_id:  491309534239885_1422202777817218
retriving comments
5 / 73 At post_id:  491309534239885_1418396001531229
retriving comments
6 / 73 At post_id:  491309534239885_1417748021596027
retriving comments
7 / 73 At post_id:  491309534239885_1414602848577211
retriving comments
8 / 73 At post_id:  491309534239885_1408867675817395
retriving comments
9 / 73 At post_id:  491309534239885_1407339839303512
retriving comments
10 / 73 At post_id:  491309534239885_1405478506156312
retriving comments
11 / 73 At post_id:  491309534239885_1402358533134976
retriving comments
12 / 73 At post_id:  491309534239885_1401260713244758
retriving comments
13 / 73 At post_id:  491309534239885_1399897003381129
retriving comments
14 / 73 At post_id:  491309534239885_1398

retriving reactions
40 / 73 At post_id:  491309534239885_1328491033855060
retriving reactions
41 / 73 At post_id:  491309534239885_1325702974133866
retriving reactions
42 / 73 At post_id:  491309534239885_1324418027595694
retriving reactions
43 / 73 At post_id:  491309534239885_1323441224360041
retriving reactions
44 / 73 At post_id:  491309534239885_1320796781291152
retriving reactions
45 / 73 At post_id:  491309534239885_1318757991495031
retriving reactions
46 / 73 At post_id:  491309534239885_1317263688311128
retriving reactions
47 / 73 At post_id:  491309534239885_1316024321768398
retriving reactions
48 / 73 At post_id:  491309534239885_1311504888887008
retriving reactions
49 / 73 At post_id:  491309534239885_1309745929062904
retriving reactions
50 / 73 At post_id:  491309534239885_1306321736071990
retriving reactions
51 / 73 At post_id:  491309534239885_1303280163042814
retriving reactions
52 / 73 At post_id:  491309534239885_1302098869827610
retriving reactions
53 / 73 At post_id

In [ ]:

retrive_facebook_page('ginmaiyood')

retriving comments
1 / 387 At post_id:  456836437854289_717442618460335
retriving comments
2 / 387 At post_id:  456836437854289_716830238521573
retriving comments
3 / 387 At post_id:  456836437854289_716647888539808
retriving comments
4 / 387 At post_id:  456836437854289_716598725211391
retriving comments
5 / 387 At post_id:  456836437854289_715939205277343
retriving comments
6 / 387 At post_id:  456836437854289_715919518612645
retriving comments
7 / 387 At post_id:  456836437854289_715776508626946
retriving comments
8 / 387 At post_id:  456836437854289_715558688648728
retriving comments
9 / 387 At post_id:  456836437854289_715411301996800
retriving comments
10 / 387 At post_id:  456836437854289_715157415355522
retriving comments
11 / 387 At post_id:  456836437854289_715012088703388
retriving comments
12 / 387 At post_id:  456836437854289_714788705392393
retriving comments
13 / 387 At post_id:  456836437854289_714636342074296
retriving comments
14 / 387 At post_id:  456836437854289_714